In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df_data = pd.read_csv("data.csv",header = None)
df_data

,0,1
0,0,156959
1,0,257228
2,0,511673
3,0,515026
4,0,935083
...,...,...
3708289,979324,540493
3708290,979324,644431
3708291,979324,816168
3708292,979324,855646


In [3]:
nodes = set(list(df_data[0]) + list(df_data[1]))
N = len(nodes)
print(N)
print(len(df_data))

979325
3708294


In [4]:
k_out = df_data[0].value_counts()
k_in = df_data[1].value_counts()
print(k_out)
print(k_in)

0
146404    4574
651712    4143
82768     3662
115692    3566
395947    3555
          ... 
110145       1
324850       1
324851       1
324852       1
66018        1
Name: count, Length: 903376, dtype: int64
1
146404    4519
395947    3783
472127    3368
357724    3308
651712    3270
          ... 
118365       1
586832       1
201087       1
321505       1
340722       1
Name: count, Length: 903792, dtype: int64


In [5]:
next_pos_map = df_data.groupby(0)[1].apply(list).to_dict()
print(next_pos_map[0])
print(next_pos_map[1])

[156959, 257228, 511673, 515026, 935083]
[9077, 83544, 104397, 191282, 258257, 350573, 451081, 635406, 866222, 897911, 941271]


In [7]:
rng = np.random.default_rng()
path = [0]
for _ in range(0,1000):
    pos = path[-1]
    if pos in next_pos_map:
        npos = next_pos_map[pos]
        path += [npos[rng.integers(0,len(npos))]]
    else:
        path += [rng.integers(0,N)]
len(path)

1001

In [8]:
df_T = df_data.copy()
df_T = df_T.merge(k_out,left_on = 0,right_on = 0)
df_T["Probability"] = 1 / df_T["count"]
df_T

,0,1,count,Probability
0,0,156959,5,0.2
1,0,257228,5,0.2
2,0,511673,5,0.2
3,0,515026,5,0.2
4,0,935083,5,0.2
...,...,...,...,...
3708289,979324,540493,10,0.1
3708290,979324,644431,10,0.1
3708291,979324,816168,10,0.1
3708292,979324,855646,10,0.1


In [10]:
import scipy.sparse as sp
row = df_T[0]
col = df_T[1]
data = df_T["Probability"]
T = sp.coo_matrix((data, (row, col)), shape=(N, N)).tocsr()


In [11]:
def f(p):
    return p*T
p = np.ones(N)/N
converged=False
for _ in range(0,1000):
    p_prev=p.copy()
    p=f(p)
    err=np.abs(p-p_prev).max()
    if err<1e-10:
        converged=True
        PageRank=pd.Series(p)
        break

print("Converged." if converged else "")

Converged.


In [12]:
PageRank.sum()

2.2795355025462517e-06

In [13]:
v_0 = np.zeros(N)
v_0[0] = 1 #i=0 のみ 1
v_24 = np.zeros(N)
v_24[24] = 1 #i=24 のみ 1
v_u = np.ones(N) / N #一様分布
print(f(v_0).sum())
print(f(v_24).sum())
print(f(v_u).sum())


1.0
0.0
0.9224476042171905


In [14]:
dangling_nodes=nodes-set(list(df_data[0]))
dangling_vec=[1 if i in dangling_nodes else 0 for i in range(0,N)]
print(dangling_vec[0])
print(dangling_vec[24])

0
1
